In [42]:
# Importação das bibliotecas necessárias
import json
import os
import logging
from jira import JIRA
from logging import config
from datetime import datetime

In [ ]:
# Função para carregar configurações do arquivo JSON
def load_config(config_file='configs/jira_configs.json'):
    if not os.path.exists(config_file):
        print(f"Arquivo de configuração '{config_file}' não encontrado. Criando arquivo com configurações padrão.")
        # Garantir que o diretório existe
        os.makedirs(os.path.dirname(config_file), exist_ok=True)
        default_config = {
            "JIRA_SERVER": "https://your_jira_server",
            "JIRA_USER": "your_username",
            "JIRA_TOKEN": "your_token",
            "PROJECT_KEY": "your_project_key",
            "FOLDER_LOGS": "."
        }
        with open(config_file, 'w') as file:
            json.dump(default_config, file, indent=4)
        return default_config
    try:
        with open(config_file, 'r') as file:
            return json.load(file)
    except json.JSONDecodeError:
        print(f"Erro ao ler o arquivo de configuração '{config_file}'. Verifique o formato JSON.")
        raise

# Carregar configurações do arquivo JSON
config = load_config(config_file='configs/jira_configs.json')

# Função para configurar logging dinamicamente
def setup_logging(log_file_path):
    # Cria o handler e o formatter para o logging
    handler = logging.FileHandler(log_file_path)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    
    # Configura o logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)  # Nível de log
    logger.addHandler(handler)
    
    return logger

# Atribuição de variáveis de configuração
JIRA_SERVER = config['JIRA_SERVER']
JIRA_USER = config['JIRA_USER']
JIRA_TOKEN = config['JIRA_TOKEN']
PROJECT_KEY = config['PROJECT_KEY']
file_path = config['FOLDER_LOGS']  # Diretório onde os arquivos de log serão salvos

# Gerar nome dinâmico para o arquivo de log com timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
log_dir = os.path.dirname(file_path)
log_file_path = os.path.join(log_dir, f"jira_check_fields_log_{timestamp}.log")

# Configuração de logging
logger = setup_logging(log_file_path)
logger.info(f"Log gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [46]:
# Conexão com o Jira
options = {'server': JIRA_SERVER}
jira = JIRA(options, basic_auth=(JIRA_USER, JIRA_TOKEN))

In [40]:
# Função para verificar e salvar os campos customizados e tipos de issue no Jira
def check_available_fields():
    try:
        project = jira.project(PROJECT_KEY)
        log_message = f"Tipos de issue disponíveis para o projeto {PROJECT_KEY}:"
        #print(log_message)
        logger.info(log_message)  # Usar o logger para registrar no arquivo de log

        # Itera sobre os tipos de issue e escreve no log
        for issue_type in project.issueTypes:
            issue_info = f"ID: {issue_type.id}, Name: {issue_type.name}"
            #print(issue_info)
            logger.info(issue_info)  # Registra no arquivo de log

        logger.info(f"Resultado salvo em: {log_file_path}")
        #print(f"Resultado salvo em: {log_file_path}")

    except Exception as e:
        logger.error(f"Erro ao verificar os tipos de issue: {str(e)}")
        #print(f"Erro ao verificar os tipos de issue: {str(e)}")

In [41]:
# Chama a função para verificar os campos
check_available_fields()